In [1]:
import numpy as np
import pandas as pd

In [2]:
from lightautoml.reader.gpu.cudf_reader import CudfReader
from lightautoml.reader.base import PandasToPandasReader

from lightautoml.transformers.base import SequentialTransformer

from lightautoml.pipelines.utils import get_columns_by_role

from lightautoml.transformers.gpu import numeric_gpu, categorical_gpu, datetime_gpu
from lightautoml.transformers import numeric, categorical, datetime

from lightautoml.tasks import Task
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML

import pandas as pd
import time
from sklearn.model_selection import train_test_split

from dask.distributed import Client
from dask_cuda import LocalCUDACluster
import cudf

In [3]:
features = pd.read_csv('../../data/small_new/multilabel/train Data.csv')
labels = pd.read_csv('../../data/small_new/multilabel/train labels.csv')
labels.drop(columns='id', inplace=True)
data = pd.concat([features, labels], axis=1)
data = data[['n_0000','n_0001','n_0002','n_0003',
             'n_0004','n_0005','n_0006','n_0007',
             'service_a', 'service_b', 'service_c', 
             'service_d', 'service_e', 'service_f',
             'service_g', 'service_h', 'service_i',
             'service_j', 'service_k', 'service_l',
             'service_m', 'service_n']]

tr_data, te_data = train_test_split(
    data, 
    test_size=0.2,  
    random_state=42
)

roles = {
    "target": {'service_a', 'service_b', 'service_c',
               'service_d', 'service_e', 'service_f',
               'service_g', 'service_h', 'service_i',
               'service_j', 'service_k', 'service_l',
               'service_m', 'service_n'},
    "drop" : ['id']
}

/tmp/ipykernel_8763/705777730.py:1: DtypeWarning: Columns (329,331,333,336,338,344,345,346,348,354,355,356,357,358,361,362,364,367,372,377,380,383,385,387,390,392,399,400,406,408,409,413,416,418,419,431,433,437,438,442,448,449,450,453,457,464,473,478,479,481,483,485,486,489,492,495,496,497,498,499,500,503,507,508,510,511,514,515,517,519,520,521,522,523,524,526,527,530,534,537,538,539,541,544,547,548,549,550,551,557,558,560,564,569,572,573,576,577,579,583,596,597,598,600,601,602,604,605,606,608,609,613,619,620,625,627,628,629,631,632,633,636,641,642,643,645,646,647,648,651,655,661,662,665,668,675,676,679,682,685,687,689,690,691,694,698,701,702,703,706,711,712,713,719,720,721,733,735,737,738,742,746,747,748,749,752,754,755,760,764,768,770,781,782,789,797,807,812,814,817,818,822,823,824,825,832,840,843,844,845,850,853,857,858,861,867,868,873,874,876,877,879,880,881,883,886,890,893,897,899,900,901,902,904,905,908,909,910,912,913,914,915,916,922,923,931,933,935,937,939,942,943,946,951,955,9

In [4]:
adv_roles = True

## Imports (for potential use)

In [5]:
# Imports from our package
from lightautoml.automl.base import AutoML

from lightautoml.automl.presets.gpu.tabular_gpu_presets import TabularAutoML_gpu, TabularUtilizedAutoML_gpu
from lightautoml.tasks import Task

from lightautoml.pipelines.features.gpu.lgb_pipeline_gpu import LGBSimpleFeatures_gpu, LGBAdvancedPipeline_gpu
from lightautoml.pipelines.features.gpu.linear_pipeline_gpu import LinearFeatures_gpu

from lightautoml.pipelines.features.lgb_pipeline import LGBSimpleFeatures, LGBAdvancedPipeline
from lightautoml.pipelines.features.linear_pipeline import LinearFeatures


from lightautoml.ml_algo.gpu.boost_cb_gpu import BoostCB_gpu
from lightautoml.ml_algo.gpu.boost_xgb_gpu import BoostXGB
#from lightautoml.ml_algo.gpu.boost_xgb_gpu import BoostXGB_dask
from lightautoml.ml_algo.gpu.linear_gpu import LinearLBFGS_gpu
#from lightautoml.ml_algo.gpu.linear_gpu import LinearL1CD_gpu
#from lightautoml.ml_algo.gpu.linear_gpu import LinearL1CD_mgpu

from lightautoml.ml_algo.boost_cb import BoostCB
#from lightautoml.ml_algo.boost_lgbm import BoostLGBM
from lightautoml.ml_algo.linear_sklearn import LinearLBFGS
from lightautoml.ml_algo.linear_sklearn import LinearL1CD


from lightautoml.pipelines.ml.base import MLPipeline
from lightautoml.pipelines.selection.importance_based import ModelBasedImportanceEstimator, ImportanceCutoffSelector

## TabularAutoML

In [6]:
task = Task('multilabel')

multilabel isn`t supported in lgb


In [7]:
automl = TabularAutoML(
    task = task, 
    timeout = 3600,
    cpu_limit = 4,
    reader_params = {'n_jobs': 4, 'cv': 3, 'random_state': 42},
    general_params = {'use_algos': [['linear_l2', 'cb']]}
)

In [8]:
oof_pred = automl.fit_predict(data, roles = roles, verbose = 2)

[10:58:37] Stdout logging level is INFO2.
[10:58:37] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[10:58:37] Task: multilabel

[10:58:37] Start automl preset with listed constraints:
[10:58:37] - time: 3600.00 seconds
[10:58:37] - CPU: 4 cores
[10:58:37] - memory: 16 GB

[10:58:37] Train data shape: (14644, 22)



/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/lightautoml/reader/guess_roles.py:56: RuntimeWarning: invalid value encountered in float_scalars
  gini_sum = a_c.sum() / a_s.sum() - (n + 1) / 2.0
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/lightautoml/reader/guess_roles.py:56: RuntimeWarning: invalid value encountered in float_scalars
  gini_sum = a_c.sum() / a_s.sum() - (n + 1) / 2.0


[10:58:41] Feats was rejected during automatic roles guess: []
[10:58:41] Layer 1 train process start. Time left 3596.22 secs


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/lightautoml/reader/guess_roles.py:56: RuntimeWarning: invalid value encountered in float_scalars
  gini_sum = a_c.sum() / a_s.sum() - (n + 1) / 2.0
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/lightautoml/reader/guess_roles.py:56: RuntimeWarning: invalid value encountered in float_scalars
  gini_sum = a_c.sum() / a_s.sum() - (n + 1) / 2.0
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/lightautoml/reader/guess_roles.py:56: RuntimeWarning: invalid value encountered in float_scalars
  gini_sum = a_c.sum() / a_s.sum() - (n + 1) / 2.0
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/lightautoml/reader/guess_roles.py:56: RuntimeWarning: invalid value encountered in float_scalars
  gini_sum = a_c.sum() / a_s.sum() - (n + 1) / 2.0


[10:58:41] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[10:58:41] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LinearL2 =====
[10:58:44] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_LinearL2 =====
[10:58:46] ===== Start working with fold 2 for Lvl_0_Pipe_0_Mod_0_LinearL2 =====
[10:58:48] Fitting Lvl_0_Pipe_0_Mod_0_LinearL2 finished. score = -7.34999771076729
[10:58:48] Lvl_0_Pipe_0_Mod_0_LinearL2 fitting and predicting completed
[10:58:48] Time left 3589.20 secs



...

[10:59:04] ===== Start working with fold 1 for Lvl_0_Pipe_1_Mod_0_CatBoost =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


[10:59:12] ===== Start working with fold 2 for Lvl_0_Pipe_1_Mod_0_CatBoost =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


[10:59:20] Fitting Lvl_0_Pipe_1_Mod_0_CatBoost finished. score = -7.322678693784647
[10:59:20] Lvl_0_Pipe_1_Mod_0_CatBoost fitting and predicting completed
[10:59:20] Time left 3557.35 secs

[10:59:20] Layer 1 training completed.

[10:59:20] Blending: optimization starts with equal weights and score -7.316042523828076
[10:59:20] Blending: iteration 0: score = -7.315802983681187, weights = [0.5839632  0.41603675]
[10:59:20] Blending: iteration 1: score = -7.315802983681187, weights = [0.5839632  0.41603675]
[10:59:20] Blending: no score update. Terminated

[10:59:20] Automl preset training completed in 43.07 seconds

[10:59:20] Model description:
Final prediction for new objects (level 0) = 
	 0.58396 * (3 averaged models Lvl_0_Pipe_0_Mod_0_LinearL2) +
	 0.41604 * (3 averaged models Lvl_0_Pipe_1_Mod_0_CatBoost) 



In [9]:
task = Task('multilabel', device='gpu')

multilabel isn`t supported in lgb
[10:59:23] CatBoost uses as obj. MultiCrossEntropy.


In [10]:
automl_gpu = TabularAutoML_gpu(
    task = task, 
    timeout = 3600,
    cpu_limit = 1,
    reader_params = {'n_jobs': 1, 'cv': 3, 'random_state': 42},
    general_params = {'use_algos': [['xgb', 'linear_l2']]}
)

In [11]:
oof_pred_gpu = automl_gpu.fit_predict(data, roles = roles, verbose = 2)

[10:59:23] Stdout logging level is INFO2.
[10:59:23] Task: multilabel

[10:59:23] Start automl preset with listed constraints:
[10:59:23] - time: 3600.00 seconds
[10:59:23] - CPU: 1 cores
[10:59:23] - memory: 16 GB

[10:59:23] Train data shape: (14644, 22)
Feats was rejected during automatic roles guess: []
[10:59:24] Layer 1 train process start. Time left 3598.37 secs
[10:59:24] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[10:59:24] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LinearL2 (orig) =====
[10:59:27] Linear model: C = 1e-05 score = -7.597445393502346
[10:59:27] Linear model: C = 5e-05 score = -7.57806301668793
[10:59:27] Linear model: C = 0.0001 score = -7.561382901053212
[10:59:28] Linear model: C = 0.0005 score = -7.507877457439777
[10:59:28] Linear model: C = 0.001 score = -7.484858495761312
[10:59:29] Linear model: C = 0.005 score = -7.434462525130588
[10:59:29] Linear model: C = 0.01 score = -7.434462525130588
[10:59:29] Linear model: C = 0.05 score = -7.4099

In [12]:
cluster = LocalCUDACluster(rmm_managed_memory=True, CUDA_VISIBLE_DEVICES="0",
                               protocol="ucx", enable_nvlink=True,
                               memory_limit="8GB")
print("dashboard:", cluster.dashboard_link)
client = Client(cluster)
client.run(cudf.set_allocator, "managed")

/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/distributed/comm/ucx.py:61: UserWarning: A CUDA context for device 0 already exists on process ID 8763. This is often the result of a CUDA-enabled library calling a CUDA runtime function before Dask-CUDA can spawn worker processes. Please make sure any such function calls don't happen at import time or in the global scope of a program.
  warnings.warn(
2022-11-20 11:01:16,553 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-y1nzf16u', purging
2022-11-20 11:01:16,559 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2022-11-20 11:01:16,559 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize


dashboard: http://127.0.0.1:8787/status


{'ucx://127.0.0.1:35889': None}

In [13]:
task = Task('multilabel', device='mgpu')

automl_mgpu = TabularAutoML_gpu(
    task = task, 
    timeout = 3600,
    cpu_limit = 1,
    reader_params = {'n_jobs': 1, 'cv': 3, 'random_state': 42, 'npartitions': 2},
    general_params = {'use_algos': [['xgb', 'linear_l2']]},
    client = client
)

oof_pred_mgpu = automl_mgpu.fit_predict(data, roles = roles, verbose = 2)

multilabel isn`t supported in lgb
[11:01:17] CatBoost uses as obj. MultiCrossEntropy.
[11:01:17] Stdout logging level is INFO2.
[11:01:17] Task: multilabel

[11:01:17] Start automl preset with listed constraints:
[11:01:17] - time: 3600.00 seconds
[11:01:17] - CPU: 1 cores
[11:01:17] - memory: 16 GB

[11:01:17] Train data shape: (14644, 22)
Feats was rejected during automatic roles guess: []
[11:01:18] Layer 1 train process start. Time left 3598.41 secs
[11:01:18] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[11:01:18] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LinearL2 (orig) =====
Score: -7.597445964813232
Score: -7.578062534332275
Score: -7.561383247375488
Score: -7.507878303527832
Score: -7.482475280761719
Score: -7.434628486633301
Score: -7.434628486633301
Score: -7.409862518310547
Score: -7.409862518310547
Score: -7.409862518310547
[11:01:22] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_LinearL2 (orig) =====
Score: -7.478099822998047
Score: -7.4584398269653

/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/distributed/worker_state_machine.py:3649: FutureWarning: The `Worker.nthreads` attribute has been moved to `Worker.state.nthreads`
  warnings.warn(
[11:01:36] task [xgboost.dask-0]:ucx://127.0.0.1:35889 got new rank 0


[11:01:58] Selector_XGB fitting and predicting completed
[11:01:58] Start fitting Lvl_0_Pipe_1_Mod_0_XGB ...
[11:01:58] ===== Start working with fold 0 for Lvl_0_Pipe_1_Mod_0_XGB (orig) =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/distributed/worker_state_machine.py:3649: FutureWarning: The `Worker.nthreads` attribute has been moved to `Worker.state.nthreads`
  warnings.warn(
[11:01:58] task [xgboost.dask-0]:ucx://127.0.0.1:35889 got new rank 0


[11:02:24] ===== Start working with fold 1 for Lvl_0_Pipe_1_Mod_0_XGB (orig) =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/distributed/worker_state_machine.py:3649: FutureWarning: The `Worker.nthreads` attribute has been moved to `Worker.state.nthreads`
  warnings.warn(
[11:02:24] task [xgboost.dask-0]:ucx://127.0.0.1:35889 got new rank 0


[11:02:49] ===== Start working with fold 2 for Lvl_0_Pipe_1_Mod_0_XGB (orig) =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/distributed/worker_state_machine.py:3649: FutureWarning: The `Worker.nthreads` attribute has been moved to `Worker.state.nthreads`
  warnings.warn(
[11:02:49] task [xgboost.dask-0]:ucx://127.0.0.1:35889 got new rank 0


[11:03:15] Lvl_0_Pipe_1_Mod_0_XGB fitting and predicting completed
[11:03:15] Time left 3482.06 secs

[11:03:15] Layer 1 training completed.

[11:03:15] Blending: Optimization starts with equal weights and score -7.368855501293354
[11:03:15] Blending, iter 0: score = -7.331661815730902, weights = [0.90983003 0.09016994]
[11:03:15] Blending, iter 1: score = -7.331661815730902, weights = [0.90983003 0.09016994]
[11:03:15] No score update. Terminated
[11:03:15] Automl preset training completed in 118.10 seconds

[11:03:15] Model description:
Final prediction for new objects (level 0) = 
	 0.90983 * (3 averaged models Lvl_0_Pipe_0_Mod_0_LinearL2) +
	 0.09017 * (3 averaged models Lvl_0_Pipe_1_Mod_0_XGB) 

